In [ ]:
# For exporting graphs as html:
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"


### Import necessary packages:

In [ ]:
import numpy as np
import sys
import os
import pandas as pd
import skimage.measure
from scipy.special import expit

from matplotlib import pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()

Import local packages

In [ ]:
all_data = True
analysis = False

from utils import ls2pc, ls2ft, calc_lin_char

### Load recording from csv file

In [ ]:
# Load csv:
file_name, lookingUp = 'rec_11c.csv', False
table = pd.read_table(file_name, delimiter=',')

table.iloc[:3]

**Note:** 
1. tick is the time column (in milliseconds)
2. stateEstimate.x and stateEstimate.y are the (x,y) coordinates of spot in meters.
3. stateEstimate.yaw is spots orientation in degrees.
4. The fields {mr18.m0, mr18.m1, ..., mr18.m15} are the range measurements in millimeters. Ranges above 4000 are measurement errors.


#### Convert recorded data into desired physical units:

In [ ]:

time = table['tick'].to_numpy()
time -= time[0]
Ts = np.median(np.diff(time, 1))
print(f'original Ts={1000.0*Ts} [ms], time is [{time[0]},{time[-1]}]')

ranges = table[[f'mr18.m{i}' for i in range(16)]].to_numpy(dtype=float)*0.001
x = table['stateEstimate.x'].to_numpy()
y = table['stateEstimate.y'].to_numpy()

# unwrap angle:
t = np.unwrap(np.deg2rad(table['stateEstimate.yaw'].to_numpy()))
print(f'ranges shape before reduction: {ranges.shape}')

#### down-sample
if needed, downsample the recording

In [ ]:

# down-sample:
D = 1
if D>1:
    ranges = skimage.measure.block_reduce(ranges, (D,1), np.max)
    print(f'ranges shape after D=({D},1) reduction: {ranges.shape}')
    x = skimage.measure.block_reduce(x, (D,), np.median)
    y = skimage.measure.block_reduce(y, (D,), np.median)
    t = skimage.measure.block_reduce(t, (D,), np.median)
    time = skimage.measure.block_reduce(time, (D,), np.max)

Ts = np.median(np.diff(time, 1))
print(f'downsampled Ts={1000.0*Ts} [ms]')

#### Plot the x,y coordinates:

In [ ]:
plt.figure(figsize=(24,16), dpi=200)
plt.plot(x[::31], y[::31], '-b.', markersize=5)
plt.show()

## Laser Scan Vs Point Cloud
*ls2pc* - converts ranges to their corresponding x,y coordinates


In [ ]:
# This is a laser scan (LS) range array:
range_ = np.array([[1,1,2,3,4,5,6,7,11,7,6,5,4,3,2,1]])
# convert the LS to point cloud, using the information about the sensor orientations relative to each other:
x_origin = np.array([0])
y_origin = np.array([0])
yaw = np.array([0])

pc_x_r_, pc_y_r_, valid_inds_bool_, valid_inds_ = ls2pc(x_origin, y_origin, yaw, range_)

# plot the point cloud
plt.figure(figsize=(24,16), dpi=100)
plt.plot(pc_x_r_, pc_y_r_, 'm.', markersize=16)
plt.plot(0, 0, '-b.', markersize=5)
plt.show()

# play arround with the other inputs of ls2pc: x_origin, y_origin, yaw

## Add the laser scanner data


In [ ]:
# Convert laser scan (16 ranges) to point cloud (2x16 xy coordinates)
pc_x_r_, pc_y_r_, valid_inds_bool_, valid_inds_ = ls2pc(x, y, t, ranges)
plt.figure(figsize=(24,16), dpi=200)
plt.plot(pc_x_r_[valid_inds_bool_][::31], pc_y_r_[valid_inds_bool_][::31], 'm.', markersize=1)
plt.plot(x[::31], y[::31], '-b.', markersize=5)
plt.show()

## Crate a database of all measurements:

In [ ]:
d = 20.0
if all_data:
    database = [\
    ('rec_08c.csv'),
    ('rec_12c.csv'),
    ('rec_11c.csv'),
    ]
    prev_i = 0
    for i, (file_name) in enumerate(database):
        table = pd.read_table(file_name, delimiter=',')

        ranges_i = table[[f'mr18.m{i}' for i in range(16)]].to_numpy(dtype=float)*0.001
        x_i = table['stateEstimate.x'].to_numpy()
        y_i = table['stateEstimate.y'].to_numpy()
        t_i = np.unwrap(np.deg2rad(table['stateEstimate.yaw'].to_numpy()))
        pc_x_r_i, pc_y_r_i, valid_inds_bool_i, valid_inds_i = ls2pc(x_i, y_i, t_i, ranges_i)
        if i>0:
            prev_i = len(x_)
            print(f'prev_i={prev_i}')
            ranges_ = np.concatenate((ranges_, ranges_i), axis=0)
            x_ = np.concatenate((x_, x_i+i*d), axis=0)
            y_ = np.concatenate((y_, y_i), axis=0)
            t_ = np.concatenate((t_, t_i), axis=0)
            pc_x_r_ = np.concatenate((pc_x_r_, pc_x_r_i+i*d), axis=1)
            pc_y_r_ = np.concatenate((pc_y_r_, pc_y_r_i), axis=1)
            valid_inds_bool_ = np.concatenate((valid_inds_bool_, valid_inds_bool_i), axis=1)
            valid_inds_ = np.concatenate((valid_inds_, valid_inds_i+prev_i), axis=0)
        else:
            ranges_ = ranges_i
            x_ = x_i
            y_ = y_i
            t_ = t_i
            pc_x_r_ = pc_x_r_i
            pc_y_r_ = pc_y_r_i
            valid_inds_bool_ = valid_inds_bool_i
            valid_inds_ = valid_inds_i

    print(f'total ranges shape: {ranges_.shape}')
    if True:
        plt.figure(figsize=(24,16), dpi=200)
        plt.plot(pc_x_r_[valid_inds_bool_][::31], pc_y_r_[valid_inds_bool_][::31], 'm.', markersize=1)
        plt.plot(x_[::31], y_[::31], '-b.', markersize=5)
        plt.show()


In [ ]:
# convert laser scans to point cloud:
pc_x_r, pc_y_r, valid_inds_bool, valid_inds = ls2pc(x, y, t, ranges)

### Plot scene using plotly instead of pyplot:

In [ ]:

mrkr = dict(color='LightSkyBlue', size=2,line=dict(color='MediumPurple', width=0.3))

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(x=pc_x_r[valid_inds_bool][::11], y=pc_y_r[valid_inds_bool][::11], name='walls', mode='markers', marker_color=valid_inds[::11], marker=mrkr),
    row=1, col=1
)
# fig.add_trace(
#     go.Scatter(x=x[::31], y=y[::31], name='trj'),
#     row=1, col=1
# )
fig.add_trace(
    go.Scatter(x=x[::31], y=y[::31], name='trj', mode='markers', marker_color=np.arange(len(x))[::31], line=dict(width=1, color='DarkSlateGrey')),
    row=1, col=1
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()




### Test total (all recordings) range measurements characteristics

In [ ]:
if all_data:
    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Scatter(y=ranges_[::21,0], name='ranges 0'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=np.convolve([1,1,1], np.diff(ranges_[:,3], axis=0)==0), name='ranges 0'),
        row=2, col=1
    )
    fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
    fig.show()

### Plot a few filtered laser scans

In [ ]:
# t0 = 125.0
t0 = 105.0
# t0 = 85.0
# t0 = 180.0
i0 = np.where(time>t0)[0][0]

rs = ranges[[i0],:]
# rs = ranges[[time_intrvl[0]],:]

sig_1 = np.roll(rs, -1)
sig_2 = np.roll(rs, 1)

sig_1_mat = np.roll(ranges, -1, axis=1)
sig_2_mat = np.roll(ranges, 1, axis=1)
sig_3_mat = np.roll(ranges, -2, axis=1)
sig_4_mat = np.roll(ranges, 3, axis=1)
# rs_med3_mat = np.median(np.concatenate((ranges, sig_1_mat, sig_2_mat), axis=0), axis=0)
print(sig_2.shape)
rs_med3 = np.median(np.concatenate((rs, sig_1, sig_2), axis=0), axis=0)
sig_3 = np.roll(rs, -2)
sig_4 = np.roll(rs, 2)
rs_med5 = np.median(np.concatenate((rs, sig_1, sig_2, sig_3, sig_4), axis=0), axis=0)

fig = make_subplots(rows=4, cols=1)  #, subplot_titles=('ranges','without odom'))
fig.add_trace(
    go.Scatter(y=rs[0,:], name='ranges'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=rs_med3, name='med3'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=rs[0,:]-rs_med3, name='diff'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=rs_med5, name='med5'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(y=rs[0,:]-rs_med5, name='diff'),
    row=4, col=1
)
fig.update_layout(width=600, title_text="local laser scan")
fig.show()

feature_1 = np.sum(np.abs(rs[0,:]-rs_med3)>0.5)
feature_2 = np.sum(np.abs(rs[0,:]-rs_med3)>1.0)
feature_3 = np.sum(np.abs(rs[0,:]-rs_med3)>1.5)

feature_3 = feature_3 - feature_2
feature_2 = feature_2 - feature_1

feature_4 = np.array(rs[0,:]>4.0, dtype=float)
feature_4 = np.sum(feature_4-np.roll(feature_4, 1)>0)

feature_5 = np.array(rs_med5>4.0, dtype=float)
feature_5 = np.sum(feature_5-np.roll(feature_5, 1)>0)

print(f'features = {feature_1}, {feature_2}, {feature_3}, {feature_4}, {feature_5}')


### Explore sigmoid function (for smoothness)

In [ ]:

def get_sigmoid(percent_loc, percent_val):
    assert(percent_val<1.0 and percent_val>0.5)
    xx = np.linspace(-10,10,500)
    yy = expit(xx)
    # print(f'shape(np.where(yy>percent_val))={np.where(yy>percent_val)[0].shape}')
    s = np.where(yy>percent_val)[0][0]
    # print(f's={s}, (xx[s]={xx[s]})')
    sf = xx[s]/percent_loc
    def sig(x):
        return expit(x*sf)
    return sig

xx = np.linspace(-10, 15,300)
sgmd_1 = get_sigmoid(5.0,0.9)
sgmd_2 = get_sigmoid(2.0,0.9)
plt.figure()
plt.plot(xx, sgmd_1(xx-5))
plt.plot(xx, 1-(sgmd_2(xx-4)+sgmd_2(-4-xx)))
plt.show()


### Check different range features:

In [ ]:
# 0.5*(sig_7_mat+ranges<1.2)+0.5*(sig_7_mat+ranges<0.9)
f_prev = lambda x: 0.5*(x<1.2)+0.5*(x<0.9)

xx = np.linspace(0, 5,300)
sgmd_close = get_sigmoid(percent_loc=0.2,percent_val=0.9)

sgmd_close_0p1 = get_sigmoid(percent_loc=0.1,percent_val=0.9)

fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_dpi(100)
fig.set_size_inches(14,4)
ax1.plot(xx, sgmd_close(1.0-xx))
ax1.plot(xx, sgmd_close(2.0-xx)-sgmd_close(1.0-xx))
ax1.plot(xx, sgmd_close(3.0-xx)-sgmd_close(2.0-xx))
ax1.plot(xx, sgmd_close(4-xx)-sgmd_close(3.0-xx))
ax1.plot(xx, f_prev(xx))

ax2.plot(xx, sgmd_close(0.5-xx))
ax2.plot(xx, sgmd_close_0p1(1.0-xx)-sgmd_close_0p1(0.5-xx))
ax2.plot(xx, sgmd_close_0p1(1.5-xx)-sgmd_close_0p1(1.0-xx))
ax2.plot(xx, sgmd_close_0p1(2.0-xx)-sgmd_close_0p1(1.5-xx))
ax2.plot(xx, f_prev(xx))
plt.show()




In [ ]:
sig_7_mat = np.roll(ranges, 8, axis=1)

sig_7 = sgmd_close(1.0-(sig_7_mat+ranges))
feature_7_mat = np.sum(sig_7, axis=1)

sig_8 = sgmd_close(2.0-(sig_7_mat+ranges))-sig_7
feature_8_mat = np.sum(sig_8, axis=1)

sig_9 = sgmd_close(3.0-(sig_7_mat+ranges))-sig_8
feature_9_mat = np.sum(sig_9, axis=1)

sig_10 = sgmd_close(4.0-(sig_7_mat+ranges))-sig_9
feature_10_mat = np.sum(sig_10, axis=1)

sig_11 = sgmd_close(5.0-(sig_7_mat+ranges))-sig_10
feature_11_mat = np.sum(sig_11, axis=1)

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5,1)
ax1.plot(feature_7_mat)
ax2.plot(feature_8_mat)
ax3.plot(feature_9_mat)
ax4.plot(feature_10_mat)
ax5.plot(feature_11_mat)
plt.show()

In [ ]:
rs_med3_mat = np.median(np.concatenate((np.expand_dims(ranges,axis=2), np.expand_dims(sig_1_mat,axis=2), np.expand_dims(sig_2_mat,axis=2)), axis=2), axis=2)
feature_4_mat = np.array(rs_med3_mat>3.0, dtype=float)
feature_4_mat = np.sum(feature_4_mat-np.roll(feature_4_mat, 1, axis=1)>0, axis=1)
sc4 = StandardScaler()
feature_4_mat = sc4.fit_transform(np.expand_dims(feature_4_mat, axis=1))
sgmd_close4 = get_sigmoid(percent_loc=0.7,percent_val=0.9)
fm4_ = sgmd_close4(rs_med3_mat-3.0)
fm4_ = np.sum(sgmd_close4(fm4_-np.roll(fm4_, 1, axis=1)-0.98), axis=1)
sc4_ = StandardScaler()
fm4_ = sc4_.fit_transform(np.expand_dims(fm4_, axis=1))
plt.figure()
plt.plot(feature_4_mat)
plt.plot(fm4_)
plt.show()


In [ ]:
fw, fw_count = calc_lin_char(pc_x_r, pc_y_r, ranges, th=0.2)
fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_dpi(100)
fig.set_size_inches(14,4)
# plt.plot(pc_x_r[valid_inds_bool__][::11], pc_y_r[valid_inds_bool__][::11], 'm.', markersize=1)
ax1.scatter(x, y, c=fw, marker='o')
ax2.scatter(x, y, c=fw_count, cmap='viridis')
plt.show()

## Perform feature calculations on all the data:

In [ ]:
def get_features(ranges, pc_x_r, pc_y_r):
    
    # shift measurements:
    sig_1_mat = np.roll(ranges, -1, axis=1)
    sig_2_mat = np.roll(ranges, 1, axis=1)
    sig_3_mat = np.roll(ranges, -2, axis=1)
    sig_4_mat = np.roll(ranges, 3, axis=1)
    sig_7_mat = np.roll(ranges, 8, axis=1)

    # calculate median 3 and 5:
    rs_med3_mat = np.median(np.concatenate((np.expand_dims(ranges,axis=2), np.expand_dims(sig_1_mat,axis=2), np.expand_dims(sig_2_mat,axis=2)), axis=2), axis=2)

    rs_med5_mat = np.median(np.concatenate((np.expand_dims(ranges,axis=2),
                                        np.expand_dims(sig_1_mat,axis=2),
                                        np.expand_dims(sig_2_mat,axis=2),
                                        np.expand_dims(sig_3_mat,axis=2),
                                        np.expand_dims(sig_4_mat,axis=2)),
                                        axis=2), axis=2)
    
    pc_x_r_matRoll1 = np.roll(pc_x_r, 1, axis=0)
    pc_x_r_matRolln1 = np.roll(pc_x_r, -1, axis=0)
    pc_y_r_matRoll1 = np.roll(pc_y_r, 1, axis=0)
    pc_y_r_matRolln1 = np.roll(pc_y_r, -1, axis=0)

    feature_2_mat =np.sum(0.5+0.5*np.tanh(4*np.multiply(np.abs(pc_x_r_matRoll1*0.5+pc_x_r_matRolln1*0.5-pc_x_r),(ranges.T<2.0))+np.multiply(np.abs(pc_y_r_matRoll1*0.5+pc_y_r_matRolln1*0.5-pc_y_r),(ranges.T<2.0))-0.3), axis=0).T

    # feature_1_mat = np.sum(np.abs(ranges-rs_med3_mat)>1.0, axis=1)
    simple_calc = False
    if simple_calc:
        sig_7 = 0.5*(sig_7_mat+ranges<1.2)+0.5*(sig_7_mat+ranges<0.9)
        feature_7_mat = np.sum(sig_7, axis=1)
    else:
        sig_7 = sgmd_close(1.0-(sig_7_mat+ranges))
        feature_7_mat = np.sum(sig_7, axis=1)

    sig_7 = sgmd_close(1.0-(sig_7_mat+ranges))
    feature_7_mat = np.sum(sig_7, axis=1)

    sig_8 = sgmd_close(2.0-(sig_7_mat+ranges))-sig_7
    feature_8_mat = np.sum(sig_8, axis=1)

    sig_9 = sgmd_close(3.0-(sig_7_mat+ranges))-sig_8
    feature_9_mat = np.sum(sig_9, axis=1)

    sig_10 = sgmd_close(4.0-(sig_7_mat+ranges))-sig_9
    feature_10_mat = np.sum(sig_10, axis=1)

    sig_11 = sgmd_close(5.0-(sig_7_mat+ranges))-sig_10
    feature_11_mat = np.sum(sig_11, axis=1)


    feature_6_mat = np.sum(0.4*(np.abs(ranges-rs_med5_mat)>1.7)+0.3*(np.abs(ranges-rs_med5_mat)>1.2)+0.3*(np.abs(ranges-rs_med5_mat)>0.5), axis=1)
    feature_3_mat = np.sum(np.logical_and(np.array(ranges<2.0,dtype=float), np.array(np.abs(0.5*sig_1_mat+0.5*sig_2_mat-ranges)<0.3,dtype=float)), axis=1)
    feature_1_mat = np.sum(np.logical_and(np.array(ranges<2.0,dtype=float), np.array(np.abs(0.25*sig_3_mat+0.25*sig_4_mat+0.25*sig_1_mat+0.25*sig_2_mat-ranges)<0.3,dtype=float)), axis=1)

    if True:
        feature_4_mat = np.array(rs_med3_mat>3.0, dtype=float)
        feature_4_mat = np.sum(feature_4_mat-np.roll(feature_4_mat, 1, axis=1)>0, axis=1)
    else:
        sgmd_close4 = get_sigmoid(percent_loc=0.7,percent_val=0.9)
        fm4_ = sgmd_close4(rs_med3_mat-3.0)
        feature_4_mat = np.sum(sgmd_close4(fm4_-np.roll(fm4_, 1, axis=1)-0.98), axis=1)


    feature_5_mat = np.array(rs_med5_mat>3.0, dtype=float)
    feature_5_mat = np.sum(feature_5_mat-np.roll(feature_5_mat, 1, axis=1)>0, axis=1)
    d1 = feature_4_mat*0.5+0.5*feature_5_mat
    d2 = feature_2_mat
    d3 = feature_1_mat

    fw, fw_count = calc_lin_char(pc_x_r, pc_y_r, ranges, th=0.2)
    feature_12_mat, feature_13_mat = fw, fw_count

    discriptor = np.concatenate((   np.expand_dims(d1,axis=1),
                                    np.expand_dims(d2,axis=1), 
                                    np.expand_dims(d3,axis=1), 
                                    np.expand_dims(feature_3_mat,axis=1),
                                    np.expand_dims(feature_6_mat,axis=1),
                                    np.expand_dims(feature_7_mat,axis=1),
                                    np.expand_dims(feature_8_mat,axis=1),
                                    np.expand_dims(feature_9_mat,axis=1),
                                    np.expand_dims(feature_10_mat,axis=1),
                                    np.expand_dims(feature_11_mat,axis=1),
                                    np.expand_dims(feature_12_mat,axis=1),
                                    np.expand_dims(feature_13_mat,axis=1)), axis=1)

    return (feature_1_mat, feature_2_mat, feature_3_mat, feature_4_mat, feature_5_mat, feature_6_mat), discriptor

(feature_1_mat, feature_2_mat, feature_3_mat, feature_4_mat, feature_5_mat, feature_6_mat), discriptor = get_features(ranges, pc_x_r, pc_y_r)
discriptor.shape

In [ ]:
# discriptor_avg3 = skimage.measure.block_reduce(discriptor, (D,1), np.mean)
D=51
scl = StandardScaler()
x_f = skimage.measure.block_reduce(x, (D,), np.median)
y_f = skimage.measure.block_reduce(y, (D,), np.median)
# iii = [1,2,3,4,5,1,2,3,4]
# jj = [1,1,1,1,1,2,2,2,2]
iii = [1,1,2,2,3,3,4,4,5]
jj = [1,2,1,2,1,2,1,2,1]

title = []
for ii in range(9):
    title.append(f"Feature {ii+1}")

fig = make_subplots(rows=5, cols=2, subplot_titles=tuple(title))
for ii, f in enumerate([feature_1_mat, feature_2_mat, feature_3_mat, feature_4_mat, feature_5_mat, feature_6_mat, discriptor[:,5], discriptor[:,6], discriptor[:,7]]):
    # c = skimage.measure.block_reduce(scl.fit_transform(np.expand_dims(feature_2_mat,axis=1)), (D,1), np.mean)
    c = skimage.measure.block_reduce(scl.fit_transform(np.expand_dims(f,axis=1)), (D,1), np.median)
    c = np.squeeze(c*10+20)


    mrkr = dict(color='LightSkyBlue', size=2,line=dict(color='MediumPurple', width=0.3))

    fig.add_trace(
        go.Scatter(x=pc_x_r[valid_inds_bool][::D], y=pc_y_r[valid_inds_bool][::D], name='walls', mode='markers', marker_color='rgba(135, 206, 250, 0.5)', marker=mrkr),
        row=iii[ii], col=jj[ii]
    )
    fig.add_trace(
        go.Scatter(x=x[::D], y=y[::D], name='trj'),
        row=iii[ii], col=jj[ii]
    )
    fig.add_trace(
        go.Scatter(x=x_f, y=y_f, name='trj', mode='markers', marker_color=c, line=dict(width=2, color="DarkSlateGrey")),
        row=iii[ii], col=jj[ii]
    )
    # fig.update_yaxes(
    #     scaleanchor="x",
    #     scaleratio=1,
    # )
fig.update_layout(width=800, height=1500, title_text='features')
fig.get_subplot(1,2)
fig.show()


In [ ]:
# if all_data:
#     for l in [ranges_, pc_x_r_, pc_y_r_]:
#         print(f'l.shape={l.shape}')
for l in [ranges, pc_x_r, pc_y_r]:
    print(f'l.shape={l.shape}')

In [ ]:
if all_data:
    _, discriptor_ = get_features(ranges_, pc_x_r_, pc_y_r_)
else:
    _, discriptor_ = get_features(ranges, pc_x_r, pc_y_r)

scaler = StandardScaler()
discriptor_scaled = scaler.fit_transform(discriptor_)

print(f'discriptor_scaled shape = {discriptor_scaled.shape}')
pca_tmp = PCA()
pca_tmp.fit(discriptor_scaled)
print(f'pca.explained_variance_ratio_={pca_tmp.explained_variance_ratio_}')

plt.figure()
plt.plot(range(1,len(pca_tmp.explained_variance_ratio_)+1), pca_tmp.explained_variance_ratio_.cumsum(),'-o')
plt.title('Explained variance components')
plt.xlabel('# components')
plt.ylabel('cumsum evr')
plt.show()


#### Keep 80% of variance ==> n_componenets == P

### Set pca parameters and fit

In [ ]:
pca_2comp = PCA(n_components=7)
discriptor_pca = pca_2comp.fit_transform(discriptor_scaled)

### Check how many clusters needed

In [ ]:
analysis = False
if analysis:
    wcss = []
    for i in range(1,18):
        kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=42)
        kmeans_pca.fit(discriptor_pca)
        wcss.append(kmeans_pca.inertia_)

    plt.figure()
    plt.plot(range(1,18), wcss, marker='o', linestyle='--')
    plt.xlabel('number of clusters')
    plt.ylabel('WCSS')
    plt.show()

##### elbow method: Need low Inertia and small number of classes
Therefore we choose M clusters

### Set KMeans parameters and fit:

In [ ]:
kmeans_pca = KMeans(n_clusters=3, init='k-means++', random_state=42)
kmeans_pca.fit(discriptor_pca)

### Implement pipe

In [ ]:

def label_data(data, pc_x_r, pc_y_r):
    _, discriptor = get_features(data, pc_x_r, pc_y_r)
    descriptor_scaled = pca_2comp.transform(scaler.transform(discriptor))

    return kmeans_pca.predict(descriptor_scaled), descriptor_scaled

# ranges.shape
labeled_ranges, descriptor_scaled = label_data(ranges, pc_x_r, pc_y_r)
print(f'shape labeled_ranges = {labeled_ranges.shape}')
print(f'labeled_ranges[:6]={labeled_ranges[:6]}')

In [ ]:


fig = make_subplots(rows=8, cols=1)
c = np.linspace(0, 1, pc_x_r.size)

fig.add_trace(
    go.Scatter(y=feature_1_mat, name='range which is avg its neighbours'),  # , marker_color=c[valid_inds]),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=feature_2_mat, name='num spikes med'),  # , marker_color=c[valid_inds]),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=feature_3_mat, name='num of colinear truplets'),  # , marker_color=c[valid_inds]),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(y=feature_4_mat, name='num openings med3'),  # , marker_color=c[valid_inds]),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(y=feature_5_mat, name='num openings med5'),  # , marker_color=c[valid_inds]),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(y=feature_4_mat*0.5+0.5*feature_5_mat, name='num openings avrg(med3,med5)'),  # , marker_color=c[valid_inds]),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(y=feature_6_mat, name='avr rng'),  # , marker_color=c[valid_inds]),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(y=discriptor[:,-2], name='opposite'),  # , marker_color=c[valid_inds]),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(y=discriptor[:,-1], name='opposite large'),  # , marker_color=c[valid_inds]),
    row=8, col=1
)
fig.update_layout(width=1000, height=800, title_text="Side By Side Subplots")
fig.show()


In [ ]:
D = 1
if D>1:
    discriptor_avg3 = skimage.measure.block_reduce(discriptor, (D,1), np.mean)
    x_avg = skimage.measure.block_reduce(x, (D,), np.mean)
    y_avg = skimage.measure.block_reduce(y, (D,), np.mean)
else:
    discriptor_avg3 = discriptor
    x_avg = x
    y_avg = y

In [ ]:

fig = make_subplots(rows=1, cols=1)
c = np.linspace(0, 1, pc_x_r.size)
fig.add_trace(
    go.Scatter(x=pc_x_r[valid_inds_bool][::21], y=pc_y_r[valid_inds_bool][::21], name='distance', mode='markers'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=x_avg[::11], y=y_avg[::11], name='distance'),
    row=1, col=1
)
D = 51
inds = np.round(len(x_avg)/51.0*np.arange(len(x_avg[::11]))/len(x_avg[::11]))
fig.add_trace(
    go.Scatter(x=x_avg[::11], y=y_avg[::11], name='distance', mode='markers', marker_color=labeled_ranges[::11]/4.0, customdata=inds, hovertemplate="<b>%{customdata}</b>"),
    row=1, col=1
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.update_layout(height=800, width=1000, title_text="Side By Side Subplots")
fig.show()

In [ ]:
import pickle

with open('data_for_infereance.pkl', 'wb') as fid:
    pickle.dump(pc_x_r, fid)
    pickle.dump(pc_y_r, fid)
    pickle.dump(valid_inds_bool, fid)
    pickle.dump(labeled_ranges, fid)
    pickle.dump(ranges, fid)
    

In [ ]:
labeled_ranges.shape

In [ ]:
from scipy import spatial

unq = np.unique(labeled_ranges)
f = lambda x: np.histogram(x, bins=len(unq), range=(unq[0]-0.5, unq[-1]+0.5))
L = len(labeled_ranges)
D = 51
x_ds = x_avg[:-(L % D)] [::D]
y_ds = y_avg[:-(L % D)] [::D]
lrs = labeled_ranges[:-(L % D)] 
if D>1:
    label_hist = np.array([f(lr)[0] for lr in list(np.reshape(lrs, (-1, D)))])
    # x_avg_2 = skimage.measure.block_reduce(x, (D,), np.mean)
    # y_avg_2 = skimage.measure.block_reduce(y, (D,), np.mean)


ll = label_hist.shape[0]
M = np.zeros((ll, ll))
for i in range(ll):
    for j in range(i,ll):
        if False:
            M[i,j] = np.linalg.norm(label_hist[i]-label_hist[j])
        else:
            M[i,j] = spatial.distance.cosine(label_hist[i],label_hist[j])
        M[j,i] = M[i,j]



In [ ]:
mm3 = go.Heatmap(
        z=M,
        # colorbar = dict(x=0.45), 
        colorscale='Viridis',
        dx=1,
        dy=1
    )

fig = make_subplots(rows=1, cols=1)  # , subplot_titles=['Exploration', 'Return'])
fig.add_trace(mm3, 1, 1)

fig.update_layout(height=800, width=800, title_text="likelyhood of same place (histogram comparing)")  #,yaxis={"title": 'time [sec]'},xaxis={"title": 'time [sec]',"tickangle": 90})
fig.show()

In [ ]:
M_x = np.expand_dims(x_ds, axis=1).T-np.expand_dims(x_ds, axis=1)
M_y = np.expand_dims(y_ds, axis=1).T-np.expand_dims(y_ds, axis=1)
Mi = np.expand_dims(np.arange(len(x_ds)), axis=1)
Mi = np.repeat(Mi, len(x_ds), axis=1)
M = np.sqrt(M_x**2+M_y**2)
ind_i, ind_j = np.where(M<0.6)
mm3 = go.Heatmap(
        z=M,
        # colorbar = dict(x=0.45), 
        colorscale='Viridis',
        dx=1,
        dy=1
    )

In [ ]:
fig = make_subplots(rows=1, cols=1)  # , subplot_titles=['Exploration', 'Return'])
fig.add_trace(
    go.Scatter(x=ind_i, y=ind_j, name='GT', mode='markers', opacity=0.5),
    row=1, col=1
)
fig.add_trace(mm3, 1, 1)

fig.update_layout(height=800, width=800, title_text="likelyhood of same place (histogram comparing)")  #,yaxis={"title": 'time [sec]'},xaxis={"title": 'time [sec]',"tickangle": 90})
fig.show()

TODO: find distance metric between classes (maybe some classes are more similar to others and require discounted penalty)

In [ ]:
if all_data:
    labeled_ranges_, descriptor_scaled_ = label_data(ranges_, pc_x_r=pc_x_r_, pc_y_r=pc_y_r_)
    fig = make_subplots(rows=1, cols=1)
    c = np.linspace(0, 1, pc_x_r.size)
    fig.add_trace(
        go.Scatter(x=pc_x_r_[valid_inds_bool_][::31], y=pc_y_r_[valid_inds_bool_][::31], name='distance', mode='markers'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_[::11], y=y_[::11], name='distance'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=x_[::11], y=y_[::11], name='distance', mode='markers', marker_color=labeled_ranges_[::11]/4.0),
        row=1, col=1
    )
    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )
    fig.update_layout(height=800, width=1000, title_text="Side By Side Subplots")
    fig.show()

In [ ]:
from numpy import matlib

def soft_clustering_weights(data, cluster_centres, **kwargs):
    
    """
    Function to calculate the weights from soft k-means
    data: Array of data. shape = N x F, for N data points and F Features
    cluster_centres: Array of cluster centres. shape = Nc x F, for Nc number of clusters. Input kmeans.cluster_centres_ directly.
    param: m - keyword argument, fuzziness of the clustering. Default 2
    """
    
    # Fuzziness parameter m>=1. Where m=1 => hard segmentation
    m = 2
    if 'm' in kwargs:
        m = kwargs['m']
    
    Nclusters = cluster_centres.shape[0]
    Ndp = data.shape[0]
    Nfeatures = data.shape[1]

    # Get distances from the cluster centres for each data point and each cluster
    EuclidDist = np.zeros((Ndp, Nclusters))
    for i in range(Nclusters):
        EuclidDist[:,i] = np.sum((data-np.matlib.repmat(cluster_centres[i], Ndp, 1))**2,axis=1)
    
    # Denominator of the weight from wikipedia:
    invWeight = EuclidDist**(2/(m-1))*np.matlib.repmat(np.sum((1./EuclidDist)**(2/(m-1)),axis=1).reshape(-1,1),1,Nclusters)
    Weight = 1./invWeight
    
    return Weight

In [ ]:
cw = soft_clustering_weights(descriptor_scaled, kmeans_pca.cluster_centers_)
confidence = np.max(cw, axis=1)>0.9

fig = make_subplots(rows=1, cols=1)
c = np.linspace(0, 1, pc_x_r.size)
fig.add_trace(
    go.Scatter(x=pc_x_r[valid_inds_bool][::21], y=pc_y_r[valid_inds_bool][::21], name='distance', mode='markers'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=x_avg[::11], y=y_avg[::11], name='distance'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x_avg[confidence][::11], y=y_avg[confidence][::11], name='distance', mode='markers', marker_color=labeled_ranges[confidence][::11]/4.0),
    row=1, col=1
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.update_layout(height=800, width=1000, title_text="Side By Side Subplots")
fig.show()

In [ ]:
w = np.max(cw, axis=1)
f = lambda x, w_: np.histogram(x, bins=4, range=(-0.5, 3.5), weights=w_)
L = len(labeled_ranges)
D = 71
lrs = labeled_ranges[:-(L % D)] 
w = w[:-(L % D)] 
w = list(np.reshape(w, (-1, D)))
if D>1:
    label_hist = np.array([f(lr, w_)[0] for lr, w_ in zip(list(np.reshape(lrs, (-1, D))), w)])
    x_avg_2 = skimage.measure.block_reduce(x, (D,), np.mean)
    y_avg_2 = skimage.measure.block_reduce(y, (D,), np.mean)


In [ ]:
ll = label_hist.shape[0]
M = np.zeros((ll, ll))
for i in range(ll):
    for j in range(i,ll):
        M[i,j] = np.linalg.norm(label_hist[i]-label_hist[j])
        M[j,i] = M[i,j]


mm3 = go.Heatmap(
        z=M,
        # colorbar = dict(x=0.45), 
        colorscale='Viridis',
        dx=2,
        dy=2
    )

fig = make_subplots(rows=1, cols=1)  # , subplot_titles=['Exploration', 'Return'])
fig.add_trace(mm3, 1, 1)
fig.update_layout(height=800, width=800, title_text="likelyhood of same place (histogram comparing)")  #,yaxis={"title": 'time [sec]'},xaxis={"title": 'time [sec]',"tickangle": 90})
fig.show()